# Building and Running the Fraud Detection Pipeline

In this notebook, we'll build and run a Kubeflow Pipeline to train our fraud detection model.

In [1]:
import sys
import os

# Add the project root directory to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust if needed
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added {project_root} to Python path")

Added /Users/prashanth.chaitanya/git-workspaces/kubeflow/kserve-example to Python path


In [2]:
# Import required libraries
import kfp
import pandas as pd
import os
import numpy as np

## Compile and Run the Pipeline

In [3]:
# Import our pipeline
from src.pipeline.pipeline import fraud_detection_pipeline

# Compile the pipeline
pipeline_func = fraud_detection_pipeline
pipeline_filename = "fraud_detection_pipeline.yaml"
kfp.compiler.Compiler().compile(pipeline_func, pipeline_filename)

print(f"Pipeline compiled to {pipeline_filename}")

Pipeline compiled to fraud_detection_pipeline.yaml


In [ ]:
from src.client.client_manager import KFPClientManager

# initialize a KFPClientManager
kfp_client_manager = KFPClientManager(
    api_url="http://localhost:8080/pipeline",
    skip_tls_verify=True,

    dex_username="user@example.com",
    dex_password="12341234",

    # can be 'ldap' or 'local' depending on your Dex configuration
    dex_auth_type="local",
)

# get a newly authenticated KFP client
# TIP: long-lived sessions might need to get a new client when their session expires
kfp_client = kfp_client_manager.create_kfp_client()

# Create an experiment
experiment_name = "fraud-detection-kserve"
experiment = kfp_client.create_experiment(name=experiment_name, namespace="kubeflow-user-example-com")

# Submit the pipeline run
run = kfp_client.create_run_from_pipeline_package(
    experiment_id=experiment.experiment_id,
    run_name="fraud-detection-training",
    pipeline_file=pipeline_filename,
    arguments={
        "model_name": "fraud-detection",
        "model_version": "v6"
    }
)

print(f"Pipeline run submitted with ID: {run.run_id}")

/Users/prashanth.chaitanya/git-workspaces/kubeflow/kserve-example/.venv/lib/python3.9/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


Pipeline run submitted with ID: 49400767-d68d-4697-87b2-16f1956b1a39


## Monitor the Pipeline Run

In [5]:
# Get pipeline run status
run_details = kfp_client.get_run(run.run_id)
print(f"Pipeline status: {run_details.state}")

Pipeline status: PENDING
